# Electronic structure through (quantum) annealing

In this project we map the electronic structure Hamiltonian to an Ising Hamiltonian and find the ground state energy.  Refer to the following references:

[1] https://arxiv.org/abs/1706.00271

[2] https://arxiv.org/abs/1811.05256

[3] https://arxiv.org/abs/1208.5986

We use molecular Hydrogen $H_2$ as an example. Assuming the atomic nucleus does not move due to its larger mass, the Hamiltonian which governs the electronic state can be transformed to a qubit representation appropriate for simulation on a quantum computer [3].  See Ref. [2], Eq. (6) for the $n$ qubit Hamiltonian, which encodes the electronic structure problem. Following Ref. [1], we then encode this problem in a classical Ising model, appropriate for annealing. This requires $r$ ancillary bit for each $n$ qubit.

The qubit Hamiltonian for moledular hydrogen $H_2$ is given by Eq. (37) in Ref. [1].  After the mapping described above, the problem eventually maps to the 2-local Ising-type Hamiltonian Eq. (41).  This goal becomes the calculation of the ground state energy of this Hamiltonian.



In [1]:
import numpy as np
from scipy.special import logsumexp

In [2]:
def energy(spins, J, h):
    # J - 2D np.array
    # h - 1D np.array
    # spins - 1D np.array (entries +/- 1)
    interaction = 0.5 * np.einsum("...i,ij,...j->...", spins, J, spins)
    field = np.einsum("...i,i->...", spins, h)
    return interaction + field

def energy_diff(i, spins, J, h):
    return -2 * np.einsum("j,...j->...", J[i, :], spins) * spins[..., i] - 2 * h[i] * spins[..., i]

In [3]:
num_spins = 10

In [4]:
# random interaction+field ising model
J = np.random.randn(num_spins, num_spins)
J = (J + J.T) / 2
for i in range(J.shape[0]):
    J[i, i] = 0

h = np.random.randn(num_spins)
spins = (2*np.random.randint(2, size=(num_spins,)) - 1)

In [5]:
# standard classical ising with no field
J = np.zeros((num_spins, num_spins))
for i in range(J.shape[0]):
    J[i, (i+1) % num_spins] = -1

J = (J + J.T)

h = np.zeros(num_spins)
spins = (2*np.random.randint(2, size=(num_spins,)) - 1)

In [6]:
def mc_step(spins, J, h, T):
    current_energy = energy(spins, J, h)
    for _ in range(spins.shape[0]):
        i = np.random.randint(spins.shape[0])        
        dE = energy_diff(i, spins, J, h)
        
        if (dE < 0) or (np.random.rand() < np.exp(-dE / T)):
            current_energy += dE
            spins[i] *= -1
        
    return spins, current_energy

In [7]:
T = 1.0
burn_in = 100
num_samples = 1000

for t in range(burn_in):
    mc_step(spins, J, h, T)
    
    
E = np.zeros(num_samples)
M = np.zeros(num_samples)

for t in range(num_samples):
    _, e = mc_step(spins, J, h, T)
    E[t] = e
    M[t] = np.abs(np.mean(spins))
    
(np.mean(E), np.std(E)/np.sqrt(num_samples)), (np.mean(M), np.std(M)/np.sqrt(num_samples))

((-7.808, 0.07372337485492643), (0.7114, 0.010786567572680384))

In [8]:
size = 10
dim = np.arange(2 ** size)
space = ((dim[:, None] & (1 << np.arange(size))) > 0)[:, ::-1]
space = 2*space.astype(int) - 1

In [9]:
E = energy(space, J, h)
M = np.abs(np.mean(space, axis=-1))

logZ = logsumexp(-E / T)
probs = np.exp(-E / T - logZ)

np.dot(E, probs), np.dot(M, probs)

(-7.955661089426349, 0.7313956631017122)